In [1]:
#import external libraries
import pandas as pd
import os
import numpy as np
import datetime
import pytz

#plotting
import matplotlib.pyplot as plt
%matplotlib notebook


#import my settings
import settings
import CleanWxData as wx
from settings import base_path

In [2]:
Glacier=settings.Glacier
Station=settings.Station

data_columns, general_data_columns, out_date_format, precip_columns, precip_gage_change, primary_temp_column, temp_columns, timezone, wind_col, wind_dir_columns = settings.get_settings(settings.Glacier, settings.Station)

print (Glacier + Station)

Gulkana1480


In [3]:
save_plots=True
save_data=True
if Glacier=='Gulkana':
    start_good_overlap_precip_date='2014-07-23'
if Glacier=='Wolverine':
    start_good_overlap_precip_date= '2013-09-13'#'2013-08-28' second is chosen by visual inspection of record

In [4]:
#Read in data
data_dir=base_path +"Data/"+Glacier+ r"/AllYears/Wx/LVL2/"
fl=Glacier.lower()+ Station + "_15min_"+"LVL2.csv"
pth=os.path.join(data_dir, fl)
dat=pd.read_csv(pth)
print("read data from "+ pth)

read data from C:\Users\ehbaker\Documents/Data/Gulkana/AllYears/Wx/LVL2/gulkana1480_15min_LVL2.csv


In [5]:
#Set time index
dat.Date=pd.to_datetime(dat.local_time, infer_datetime_format=True)
dat=dat.set_index('local_time')

#Rename best daily temperature column
dat.rename(columns={'site_temp_USGS':'Temp_USGS'}, inplace=True)
new_columns = [w.replace('site_temp', 'Temp') for w in dat.columns]
dat.columns=new_columns

C:\Users\ehbaker\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [6]:
#Reset naming convention to match expected in this slightly older script
new_columns=[w.replace('site_temp', 'Temp') for w in dat.columns]
dat.columns=new_columns
dat.tail(1)

,Temp,Tpassive1,Tpassive2,TAspirated1,TAspirated2,TPGIncremental,StageIncremental,RelHum,WindSpeed,WindGustSpeed,WindDir,VecAvgWindDir,TPGCumulative,StageCumulative
local_time,,,,,,,,,,,,,,
2020/05/01 19:15,-3.2,-3.4,NaN,-3.2,-3.5,-1.0,NaN,NaN,2.0,3.4,NaN,249.0,3062.0,NaN


In [7]:
#Determine if precip, temperature, and wind data are present
precip_present=len(precip_columns)>0
temp_present=len(temp_columns)>0
wind_present=len(wind_col)>0

In [8]:
# #Read in raw 15 minute data
raw_data_path=base_path+ 'Data/' + Glacier+ '/AllYears/Wx/LVL0/telemetry_added/' + Glacier.lower() + Station +'_15min_all_LVL0.csv'
if os.path.isfile(raw_data_path):
    raw_dat=pd.read_csv(raw_data_path, infer_datetime_format=True)
    raw_dat['local_time']=pd.to_datetime(raw_dat.local_time)
    raw_dat.local_time[0]-pd.Timedelta('12 hours') #THIS IS FOR DISPLAY PLOTTING PURPOSES ONLY
    raw_dat=raw_dat.set_index('local_time') 

In [9]:
cols=list(set(dat.columns) -set(['Temp_min', 'Temp_max', 'Temp_WMO', 'Temp_USGS']))

In [10]:
temp_cols2=[x +'_USGS' for x in temp_columns]
dat[temp_cols2]['2018':'2019'].plot()

KeyError: "None of [Index(['Tpassive1_USGS', 'Tpassive2_USGS', 'TAspirated1_USGS',\n       'TAspirated2_USGS'],\n      dtype='object')] are in the [columns]"

In [11]:
# #Plot all the data
# fig = plt.figure(figsize=(16,10))
# ax = fig.add_subplot(111)
# cols=cols
# dat[cols].plot(ax=ax, subplots=True, layout=(len(cols),1), sharex=True) # ax=ax points df.plot to fig
# for each in fig.axes: # You can still modify these axes!
#     each.legend(loc='center left', bbox_to_anchor=(1, 0.8))
# plt.subplots_adjust(right=0.8)
# plt.savefig(r'Q:\Project Data\GlacierData\Benchmark_Program\Data/' + Glacier+r'\AllYears\Wx\LVL2/LVL2S' + Glacier +Station+'WxData.jpg')

In [12]:
#Give names of temperature and precip columns in level 2 data (different than original input)
temp_col='Temp_USGS'
temp_maxmin_cols=['Temp_min', 'Temp_max']
precip_cols=['Stage_Precip', 'TPG_Precip']

In [13]:
#Read in historical daily data (from analog strip charts)
GUI_dat=pd.read_csv(base_path.split('/Ben')[0]+ '/Matlab_Files/mb_GUI/Archive/2017.12.14/data/' + Glacier+ '/Input/Input_' + Glacier + '_Daily_Weather.csv', infer_datetime_format=True)
GUI_dat['date']=pd.to_datetime(GUI_dat.Date, format="%m/%d/%Y")
GUI_dat=GUI_dat.set_index('date')
GUI_dat.rename(columns = {'Temperature':'temp_original', 'Precipitation':'precip_original'}, inplace = True)

dat_new=dat.copy() #save data w/o older added
start_date_15min_data=dat_new.index[0] #this is the date that we have 15-minute data
start_historic_data=GUI_dat.first_valid_index()

#Determine if historic program data from the GUI input from pre-digital era should be appended here
need_to_append_historical_program_data=start_historic_data<start_date_15min_data
if (Glacier+Station=='Gulkana1725')| (Glacier + Station=='Wolverine1420'):
    need_to_append_historical_program_data=False
print (need_to_append_historical_program_data)
print (start_date_15min_data)

FileNotFoundError: [Errno 2] File b'C:\\Users\\ehbaker\\Documents//Matlab_Files/mb_GUI/Archive/2017.12.14/data/Gulkana/Input/Input_Gulkana_Daily_Weather.csv' does not exist: b'C:\\Users\\ehbaker\\Documents//Matlab_Files/mb_GUI/Archive/2017.12.14/data/Gulkana/Input/Input_Gulkana_Daily_Weather.csv'

In [14]:
Glacier

'Gulkana'

In [15]:
dat_new.Stage_Precip.first_valid_index()

NameError: name 'dat_new' is not defined

### Create single continuous precipitation record
#####  First - calculate the scale factor that should be applied to bias-adjust record thru the stage gage -> TPG transition

In [16]:
#Define precip phase
daily_dat=wx.define_precip_phase(dat, temp_col) #daily dat is a copy of original; original dat remains unedited

KeyError: 'Temp_USGS'

In [17]:
temp_col

'Temp_USGS'

In [18]:
dat.head()

,Temp,Tpassive1,Tpassive2,TAspirated1,TAspirated2,TPGIncremental,StageIncremental,RelHum,WindSpeed,WindGustSpeed,WindDir,VecAvgWindDir,TPGCumulative,StageCumulative
local_time,,,,,,,,,,,,,,
2018/08/01 00:15,14.1,14.9,NaN,14.1,14.6,0.0,NaN,58.0,2.1,NaN,NaN,39.0,426.0,NaN
2018/08/01 00:30,14.3,15.5,NaN,14.3,14.6,0.0,NaN,58.0,2.2,3.6,NaN,50.0,426.0,NaN
2018/08/01 00:45,13.4,14.5,NaN,13.4,13.7,0.0,NaN,59.0,4.3,6.5,NaN,37.0,426.0,NaN
2018/08/01 01:00,13.1,13.9,NaN,13.1,13.3,0.0,NaN,62.0,4.8,6.5,NaN,42.0,426.0,NaN
2018/08/01 01:15,13.1,14.0,NaN,13.1,13.7,0.0,NaN,61.0,4.7,6.3,NaN,42.0,426.0,NaN


In [19]:
#Calculate precip undercatch; ONLY for purpose of deriving scale factor
if wind_present and (len(precip_columns)>1):
    daily_dat=wx.correct_undercatch_yang98(daily_dat, precip_cols[0], 'phase', wind_col)
    daily_dat=wx.correct_undercatch_yang98(daily_dat, precip_cols[1], 'phase', wind_col)

NameError: name 'daily_dat' is not defined

In [20]:
if precip_gage_change:
    daily_dat[precip_cols][start_good_overlap_precip_date: daily_dat[precip_cols[0]].last_valid_index()].cumsum().plot()
    ratiodf=daily_dat[precip_cols][start_good_overlap_precip_date: daily_dat[precip_cols[0]].last_valid_index()].cumsum()[-1:]
    scale_factor=(ratiodf.TPG_Precip/ratiodf.Stage_Precip).values[0]
    print("Scale factor = " + str(scale_factor))
    
    adjusted_stage=daily_dat.Stage_Precip.copy()*scale_factor
    adjusted_stage[start_good_overlap_precip_date: daily_dat[precip_cols[0]].last_valid_index()].cumsum().plot(color='red')

NameError: name 'daily_dat' is not defined

### Bias-adjust stage gage data to match TPG

In [21]:
if precip_gage_change:
    #Merge older pre-digital data with new 15-min derived daily data
    dat=GUI_dat[['temp_original', 'precip_original']].merge(dat[[temp_col]+ precip_cols+[wind_col]+temp_maxmin_cols], left_index=True, right_index=True, how='outer')

    #Bias adjustment
    dat['Precipitation']=pd.np.nan
    dat['stage_adjusted']=dat.Stage_Precip*scale_factor #Adjusted by factor of 5 earlier to account for orifice size, and *2 for wind factor (empirical)
    dat['original_adjusted']=dat.precip_original*scale_factor
    #Create full record from 3 pieces
    dat.loc[start_good_overlap_precip_date:,'Precipitation']=dat.loc[start_good_overlap_precip_date:,'TPG_Precip'] #precip is TPG from install -> end
    dat.loc[dat.stage_adjusted.first_valid_index():start_good_overlap_precip_date,'Precipitation']=dat.loc[dat.stage_adjusted.first_valid_index():start_good_overlap_precip_date,'stage_adjusted']#precip is adjusted stage (made from 15 min) back to late 2003
    dat.loc[:dat.stage_adjusted.first_valid_index(),'Precipitation']=dat.loc[:dat.stage_adjusted.first_valid_index(),'original_adjusted']#precip is adjusted stage (made from 15 min) back to late 2003
    #How does this look for a continuous precip record
    plt.figure(figsize=(8,6))
    dat.Precipitation.cumsum().plot(color='black')
    plt.axvline(x=start_good_overlap_precip_date, color='c', label="TPG Installed")
    plt.axvline(x=dat.stage_adjusted.first_valid_index(), color='darkmagenta',label="15 min data available")
    plt.legend()
    if save_plots:
        strFile = './figs/' + Glacier+Station+ '/precip_match/' + Glacier + Station+ '_Different_Eras_Of_Precip_Record.jpg'
        if os.path.isfile(strFile):
           os.system("rm "+strFile)
        plt.savefig(strFile)

NameError: name 'GUI_dat' is not defined

In [22]:
need_to_append_historical_program_data

NameError: name 'need_to_append_historical_program_data' is not defined

In [23]:
#Create single continuous temperature record
dat['Temperature']=dat[temp_col]
if need_to_append_historical_program_data:
    dat.loc[(dat.Temp_USGS.isnull()) & (dat.index<start_date_15min_data), 'Temperature']=dat.loc[(dat.Temp_USGS.isnull()) & (dat.index<start_date_15min_data), 'temp_original']

KeyError: 'Temp_USGS'

In [35]:
dat

,Temp,Tpassive1,Tpassive2,TAspirated1,TAspirated2,TPGIncremental,StageIncremental,RelHum,WindSpeed,WindGustSpeed,WindDir,VecAvgWindDir,TPGCumulative,StageCumulative
local_time,,,,,,,,,,,,,,
2018/08/01 00:15,14.1,14.9,NaN,14.1,14.6,0.0,NaN,58.0,2.1,NaN,NaN,39.0,426.0,NaN
2018/08/01 00:30,14.3,15.5,NaN,14.3,14.6,0.0,NaN,58.0,2.2,3.6,NaN,50.0,426.0,NaN
2018/08/01 00:45,13.4,14.5,NaN,13.4,13.7,0.0,NaN,59.0,4.3,6.5,NaN,37.0,426.0,NaN
2018/08/01 01:00,13.1,13.9,NaN,13.1,13.3,0.0,NaN,62.0,4.8,6.5,NaN,42.0,426.0,NaN
2018/08/01 01:15,13.1,14.0,NaN,13.1,13.7,0.0,NaN,61.0,4.7,6.3,NaN,42.0,426.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020/05/01 18:15,0.1,-1.3,NaN,0.1,-0.9,0.0,NaN,NaN,1.1,1.5,NaN,122.0,3062.0,NaN
2020/05/01 18:30,-0.7,-2.5,NaN,-0.7,-1.8,1.0,NaN,NaN,1.8,2.7,NaN,138.0,3063.0,NaN
2020/05/01 18:45,-1.2,0.1,NaN,-1.2,-2.4,0.0,NaN,NaN,0.9,2.1,NaN,142.0,3063.0,NaN


In [24]:
dat[['Temperature', 'Precipitation']].plot()

KeyError: "None of [Index(['Temperature', 'Precipitation'], dtype='object')] are in the [columns]"

## Save data

In [25]:
#Save file with all columns
save_pth = base_path + '/Data/' + Glacier + '/AllYears/Wx/LVL3/AllColumns_' + Glacier + Station+ '.csv'
dat.to_csv(save_pth)
dat.dropna(how='all').tail()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ehbaker\\Documents//Data/Gulkana/AllYears/Wx/LVL3/AllColumns_Gulkana1480.csv'

In [26]:
if Glacier +Station in ['Wolverine990', 'Gulkana1480']:
    if save_data:
        save_dat=dat[['Temperature', 'Precipitation']]
        save_dat.loc[:,'Temperature']=["%.1f" %x for x in save_dat.Temperature]
        save_dat.loc[:,'Precipitation']=["%.1f" %x for x in save_dat.Precipitation]
        save_dat.index.name='Date'
        save_dat['Elevation']=Station
        save_pth=base_path + '/Data/' + Glacier + '/AllYears/Wx/LVL3/Input_' + Glacier +'_Daily_Weather.csv'
        save_dat.to_csv(save_pth, float_format='%g', date_format='%Y/%m/%d') #write select data

KeyError: "None of [Index(['Temperature', 'Precipitation'], dtype='object')] are in the [columns]"

In [27]:
#Define precip phase
dat=wx.define_precip_phase(dat, 'Temperature')

KeyError: 'Temperature'

In [28]:
#Adjust recent data by wind speed, and save just this recent adjusted data
if wind_present:
    dat_recent=wx.define_precip_phase(dat[dat.WindSpeed.first_valid_index():].copy(), 'Temperature') #daily dat is a copy of original; original dat remains unedited

    #Calculate precip undercatch, using CONSTANT yearly pattern of wind speed
    dat_recent=wx.correct_undercatch_yang98(dat_recent, 'Precipitation', 'phase', 'WindSpeed', inplace=False)
    dat_recent[['Precipitation', 'Precipitation_undercatch_adj']].cumsum().plot()

    #Save this adjusted data
    if save_data:
        save_pth=base_path + '/Data/' + Glacier + '/AllYears/Wx/LVL3/OnlyRecentRealWindUndercatch_Adj' + Glacier + '_Daily_Weather.csv'
        dat_recent[dat.WindSpeed.first_valid_index():].to_csv(save_pth, float_format='%g', date_format='%Y/%m/%d') #write select data

KeyError: 'Temperature'

### Calculate average wind-speeds, and adjust precipitation by this factor

In [29]:
#Stuff to make period-of-record average for Wind Speed!
#Create period-of-record daily average (e.g. avg. on Mar1, Jan3, etc.)
if wind_present:
    n_window=30
    period_of_record_avg_ws=pd.DataFrame(dat[wind_col].groupby([dat.index.month, dat.index.day]).mean())
    period_of_record_avg_ws=period_of_record_avg_ws.reset_index()
    period_of_record_avg_ws.columns=['month', 'day', wind_col]

    period_of_record_avg_ws.index.name='DOY'
    period_of_record_avg_ws=period_of_record_avg_ws.reset_index()

    period_of_record_avg_ws_r30=period_of_record_avg_ws[wind_col].rolling(n_window, center=True).mean()

    #Calculate avg wind speed for periods before and after the moving-window won't b/c DOY 1-365 disconnect
    extra_ws=period_of_record_avg_ws.copy()
    extra=extra_ws[wind_col][0:n_window]
    extra_for_movavg=period_of_record_avg_ws.WindSpeed.append(extra, ignore_index=True)
    long_windowavg=extra_for_movavg.rolling(n_window, center=True).mean()
    period_of_record_avg_ws_r30[0:int(n_window/2)]=long_windowavg[366:(366+int(n_window/2))] #replace missing portion at beginning
    period_of_record_avg_ws_r30[int(366-(n_window/2)):366]=long_windowavg[366:int(366+(n_window/2))] #replace missing portion at end

    #Make Plot of this
    plt.figure()
    period_of_record_avg_ws.index.name='DOY'
    ax=period_of_record_avg_ws['WindSpeed'].plot()
    period_of_record_avg_ws_r30.plot(ax=ax, color='red')
    plt.legend(['Wind Speed', '30-day average'])
    plt.title('Period of Record Average Wind Speed')
    plt.tight_layout()
    plt.savefig('./figs/' + Glacier + Station+ '/trends/' + Glacier + Station+ 'PeriodOfRecordWindSpeed.jpg')

    #Add wind speed on day of year to main dataframe
    dat['DOY']=dat.index.dayofyear-1
    period_of_record_avg_ws_r30.index.name="DOY"
    period_of_record_avg_ws_r30.name='WS_Constant'
    period_of_record_avg_ws_r30=period_of_record_avg_ws_r30.reset_index()

AttributeError: 'Index' object has no attribute 'month'

In [30]:
if wind_present:
    small_dat=dat[['Precipitation', 'Temperature', 'DOY', 'WindSpeed', 'phase']+ temp_maxmin_cols].copy()
    small_dat.index.name='Date'
    small_dat.reset_index(inplace=True)
    dat_adj=small_dat.merge(period_of_record_avg_ws_r30, how='left', left_on='DOY', right_on='DOY')

    dat_adj=dat_adj.set_index('Date')

    #Calculate precip undercatch, using CONSTANT yearly pattern of wind speed
    dat_adj=wx.correct_undercatch_yang98(dat_adj, 'Precipitation', 'phase', 'WS_Constant', inplace=False)
    dat_adj[['Precipitation', 'Precipitation_undercatch_adj']].cumsum().plot()

    #Rename this column
    dat_adj.rename(columns={'Precipitation_undercatch_adj':'Precip_AvgWindSpeed_UndercatchAdj'}, inplace=True)

    #Make another, using the REAL daily wind speed
    dat_adj=wx.correct_undercatch_yang98(dat_adj, 'Precipitation', 'phase', 'WindSpeed', inplace=False)
    dat_adj.rename(columns={'Precipitation_undercatch_adj':'Precip_MeasuredWindSpeed_UndercatchAdj'}, inplace=True)
    #Set to NAN before wind speed measured (in fucntion, NANs are filled to keep precip where recorded but wind down)
    dat_adj.loc[:dat_adj.WindSpeed.first_valid_index(),'Precip_MeasuredWindSpeed_UndercatchAdj']=pd.np.nan

KeyError: "['phase', 'Temp_min', 'DOY', 'Temp_max', 'Precipitation', 'Temperature'] not in index"

In [31]:
if wind_present:
    #Save this adjusted data
    save_pth=base_path + '/Data/' + Glacier + '/AllYears/Wx/LVL3/Undercatch_Adj' + Glacier + '_Daily_Weather.csv'
    dat_adj.to_csv(save_pth, float_format='%g', date_format='%Y/%m/%d') #write select data
else:
    dat_adj=dat.copy()
    save_pth=base_path + '/Data/' + Glacier + '/AllYears/Wx/LVL3/Undercatch_Adj' + Glacier + '_Daily_Weather.csv'
    dat_adj.to_csv(save_pth, float_format='%g', date_format='%Y/%m/%d') #write select data

    print("no wind to adjust for undercatch")

NameError: name 'dat_adj' is not defined

In [32]:
if wind_present:
    precip_col='Precip_AvgWindSpeed_UndercatchAdj'
else:
    precip_col='Precipitation'
dat_adj['snow']=pd.np.nan #dat_adj.loc[dat_adj.phase=='snow', precip_col]
dat_adj['rain']=pd.np.nan #dat_adj.loc[dat_adj.phase=='rain', precip_col]

dat_adj['scale_factor_for_mixed_Fsnow']=pd.np.nan
dat_adj.loc[dat_adj.phase=='mixed', 'scale_factor_for_mixed_Fsnow']=((dat_adj.loc[dat_adj.phase=='mixed', 'Temperature'])/-4 + 0.75)
dat_adj.loc[dat_adj.phase=='rain', 'scale_factor_for_mixed_Fsnow']=0
dat_adj.loc[dat_adj.phase=='snow', 'scale_factor_for_mixed_Fsnow']=1

dat_adj['snow']=dat_adj[precip_col]*dat_adj.scale_factor_for_mixed_Fsnow

dat_adj['rain']=dat_adj[precip_col]*(1-dat_adj.scale_factor_for_mixed_Fsnow)

dat_adj['fSnow']=dat_adj.snow/dat_adj[precip_col]

NameError: name 'dat_adj' is not defined

In [33]:
#add numeric date
dat_adj['decimal_date']=dat_adj.index.year+ (dat_adj.index.dayofyear -1)/365

NameError: name 'dat_adj' is not defined

In [34]:
save_pth=base_path + '/Data/' + Glacier + '/AllYears/Wx/LVL3/RainSnowFractionData_' + Glacier + Station+ '.csv'
save_cols=['Precipitation', 'Temperature','phase','snow', 'rain','decimal_date']
if wind_present:
    save_cols=save_cols+['WindSpeed', 'WS_Constant', 'Precip_AvgWindSpeed_UndercatchAdj', 'Precip_MeasuredWindSpeed_UndercatchAdj']
dat_adj[save_cols].to_csv(save_pth)

NameError: name 'dat_adj' is not defined

In [34]:
#dat_adj_og=dat_adj.copy()
#dat_adj=dat_adj_og.copy()

In [35]:
# #Aggregate monthly data & plot

# #FIRST! Subset to only winter months (following Harpold 2012 trend paper)
# dat_adj['month']=dat_adj.index.month
# dat_adj=dat_adj[dat_adj.month.isin([6,7,8])]#isin([11,12,1,2,3])]


# mth_dat=pd.DataFrame()
# mth_dat['snow']=wx.aggregate_time_with_threshold(dat_adj['snow'], 'M', func='sum', steps_in_period=30)
# mth_dat['rain']=wx.aggregate_time_with_threshold(dat_adj['rain'], 'M', func='sum', steps_in_period=30)
# mth_dat['Precipitation']=wx.aggregate_time_with_threshold(dat_adj['Precipitation'], 'M', func='sum', steps_in_period=30)

# #calculate snow fraction
# mth_dat['fSnow']=mth_dat['snow']/(mth_dat['rain']+mth_dat['snow'])

# #Add decimal date
# mth_dat['decimal_date']=mth_dat.index.year+ (mth_dat.index.dayofyear -1)/365

In [36]:
# mth_dat.tail()
# import plot_funcs

In [37]:
# plot_funcs.OLS_plot('decimal_date', 'Precipitation', dat=mth_dat.dropna(subset=['Precipitation']), title='Monthly '+ 'fSnow, Nov-Mar', aspect=2.5)
# plt.savefig('./figs/trends/' + Glacier + Station+ '_Monthly_fSnowTrend_WINTER' +'.jpg')

In [38]:
# dat_adj.tail()

In [39]:
# #Aggregate yearly data & plot

# dat_adj['month']=dat_adj.index.month
# dat_adj=dat_adj[dat_adj.month.isin([11,12,1,2,3])]

# yr_dat=pd.DataFrame()
# yr_dat['snow']=wx.aggregate_time_with_threshold(dat_adj['snow'], 'AS', func='sum', steps_in_period=150)
# yr_dat['rain']=wx.aggregate_time_with_threshold(dat_adj['rain'], 'AS', func='sum', steps_in_period=150)

# #calculate snow fraction
# yr_dat['fSnow']=yr_dat['snow']/(yr_dat['rain']+yr_dat['snow'])

# #Add decimal date
# yr_dat['decimal_date']=yr_dat.index.year+ (yr_dat.index.dayofyear -1)/365


# plot_funcs.OLS_plot('decimal_date', 'fSnow', dat=yr_dat.dropna(subset=['fSnow']), title='Yearly '+ 'fSnow', aspect=2.5)
# #plt.savefig('./figs/trends/' + Glacier + Station+ '_Yearly_fSnowTrend' +'.jpg')

Trend Analysis

In [40]:
# import plot_funcs
# import CleanWxData
# import seaborn as sns
# import statsmodels.api as sm
# sns.set_palette("Set2")
# sns.set(style='whitegrid') #set seaborn figure size to A4 paper

In [41]:
# dat['decimal_date']=dat.index.year+ (dat.index.dayofyear -1)/365

In [42]:
dat.head(1)

,temp_original,precip_original,Temp_USGS,Stage_Precip,TPG_Precip,WindSpeed,Temp_min,Temp_max,Precipitation,stage_adjusted,original_adjusted,Temperature,phase,DOY
Date,,,,,,,,,,,,,,
1964-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274


In [43]:
dat.columns

Index(['temp_original', 'precip_original', 'Temp_USGS', 'Stage_Precip',
       'TPG_Precip', 'WindSpeed', 'Temp_min', 'Temp_max', 'Precipitation',
       'stage_adjusted', 'original_adjusted', 'Temperature', 'phase', 'DOY'],
      dtype='object')

In [44]:
# #Simple  OLS plot of old vs new precip
# #var='Precipitation'
# var='Temperature'
# if var=='Temperature':
#     var_old='temp_original'
#     agg_func='mean'
# if var=='Precipitation':
#     var_old='precip_original'
#     agg_func='sum'
# sns.set(rc={'figure.figsize':(12, 6)}, style='whitegrid') #set seaborn figure size to A4 paper
# fig=plot_funcs.OLS_plot(var_old, var, dat=dat.dropna(subset=[var]), title='Daily '+ var, aspect=2.5)
# if save_plots:
#     plt.savefig('./figs/precip_match/' + Glacier + Station+ '_Compare2preciprecords_'+ var +'.jpg')

In [45]:
# #Aggregate monthly data
# mth_dat=pd.DataFrame()
# mth_dat[var]=wx.aggregate_time_with_threshold(dat[var], 'M', func=agg_func, steps_in_period=30)

# #Add decimal date
# mth_dat['decimal_date']=mth_dat.index.year+ (mth_dat.index.dayofyear -1)/365

In [46]:
# print(len(mth_dat[var]['1967':][mth_dat[var]['1967':].isnull()]))

# len(mth_dat[var]['1967':][mth_dat[var]['1967':].notnull()])

In [47]:
# #Simple  OLS plot of monthly data
# sns.set(rc={'figure.figsize':(6, 12)}, style='whitegrid') #set seaborn figure size to A4 paper
# fig=plot_funcs.OLS_plot('decimal_date', var, dat=mth_dat.dropna(subset=[var]), title='Monthly '+ var + "; "+ agg_func, aspect=2.5)
# #if save_plots:
# plt.savefig('./figs/trends/' + Glacier + Station+ '_OLS_MonthlyTrend21_'+ var +'.jpg')

In [48]:
# #Aggregate to yearly data
# y_dat=pd.DataFrame()
# y_dat[var]=wx.aggregate_time_with_threshold(dat[var], 'AS', func=agg_func, steps_in_period=365)
# #Add decimal date
# y_dat['decimal_date']=y_dat['decimal_date']=y_dat.index.year+ (y_dat.index.dayofyear -1)/365

# #y_dat.loc['1978':'1981', 'Precipitation']=pd.np.nan #see what happens w/o 1980 PDO bump

# #Simple  OLS plot
# sns.set(rc={'figure.figsize':(12, 6)}, style='whitegrid') #set seaborn figure size to A4 paper
# fig=plot_funcs.OLS_plot('decimal_date', var, dat=y_dat.dropna(subset=[var]), title='Yearly '+ var + "; "+ agg_func, aspect=2.5)
# if save_plots:
#     plt.savefig('./figs/trends/' + Glacier + Station+ '_OLS_Annual_2'+ var +'.jpg')

In [49]:
# mth_dat['month']=mth_dat.index.month
# mth_dat['year']=mth_dat.index.year
# g=sns.factorplot(x='year', y=var, hue='month', row='month', data=mth_dat, kind='point', aspect=5)
# plt.xticks(rotation=-90)
# if save_plots:
#     plt.savefig('./figs/trends/' + Glacier + Station+ '_ScatterByMonthThruTime_'+ var +'.jpg')

In [50]:
# plt.figure()
# #sns.set(rc={'figure.figsize':(12, 6)}) #set seaborn figure size to A4 paper
# g=sns.boxplot(x=mth_dat.index.year, y=var, data=mth_dat)
# plt.xticks(rotation=-90)

In [51]:
# #Find missing values and fill
# var_filled=dat[var].interpolate()
# var_filled=var_filled[var_filled.first_valid_index():]

# #Deal with adding dates to x-axis
# jan1s=var_filled.index[var_filled.index.dayofyear==1]
# jan1_labels=jan1s.year
# idx_jan1s=[var_filled.index.get_loc(x) for x in jan1s]


In [52]:
# #Make decomposition plot
# decomposition= sm.tsa.seasonal_decompose(var_filled.values, model='additive', freq=365) #frequency is yearly, but must be integer
# fig = decomposition.plot()
# my_xticks=jan1_labels
# plt.xticks(idx_jan1s, my_xticks)
# plt.show()
# fig.autofmt_xdate()
# plt.savefig("./figs/seasonal_trend_decompositon_3"+ var+ "_"+ Glacier + Station+ ".jpg")

In [53]:
# df=pd.DataFrame(decomposition.trend)
# df.columns=['trend']

In [54]:
# df=df.reset_index()

In [55]:
# df['year']=df.index/365

In [56]:
# plot_funcs.OLS_plot("year", "trend", df)

In [57]:
import seaborn as sns
#Colored by date
f, ax = plt.subplots()
#cmap = sns.cubehelix_palette(as_cmap=True)
cmap=sns.diverging_palette(220, 20, n=7, as_cmap=True)
points = ax.scatter(dat.precip_original, dat.Precipitation, c=dat.index.year, s=50, cmap=cmap, alpha=0.7)
f.colorbar(points)
plt.xlabel="Original Precip"
plt.ylabel="New Precip"
if save_plots:
    plt.savefig('./figs/precip_match/' + Glacier + Station+ '_GUI vs New Colored by Year2.jpg')

<IPython.core.display.Javascript object>

FileNotFoundError: [Errno 2] No such file or directory: './figs/precip_match/Gulkana1480_GUI vs New Colored by Year2.jpg'

In [ ]:
dat[['Precipitation', 'precip_original']].plot()

In [ ]:
#save_dat.to_csv(save_pth, float_format='%g', date_format='%Y/%m/%d') #write select data

In [ ]:
dat_adj['ratio']=dat_adj.Precipitation_undercatch_adj/dat_adj.Precipitation

In [ ]:
plt.figure()
dat_adj.ratio.plot(kind='hist', bins=50)

In [ ]:
#Aggregate Ratio yearly data
var='ratio'
y_dat=dat_adj.ratio.resample('AS', convention='start').agg(['median', 'count'])

#Add decimal date
y_dat['decimal_date']=y_dat['decimal_date']=y_dat.index.year+ (y_dat.index.dayofyear -1)/365

#y_dat.loc['1978':'1981', 'Precipitation']=pd.np.nan #see what happens w/o 1980 PDO bump

#Simple  OLS plot
sns.set(rc={'figure.figsize':(6, 6)}, style='whitegrid') #set seaborn figure size to A4 paper
fig=plot_funcs.OLS_plot('decimal_date', 'median', dat=y_dat.dropna(subset=['median']), title='Precip Adjustment Ratio', aspect=2.5)
# if save_plots:
#     plt.savefig('./figs/trends/' + Glacier + Station+ '_trendInThePrecipAdjustmentFactor'+ var +'.jpg')

In [ ]:
dat.join(dat_adj, t)

In [ ]:
dat_adj.ratio.max()

In [ ]:
dat_adj[var]

In [ ]:
#Aggregate to yearly data
var='ratio'
#y_dat=dat_adj.ratio.resample('AS', convention='start').agg(['sum', 'count'])
y_dat=pd.DataFrame()
y_dat[var]=wx.aggregate_time_with_threshold(dat_adj[var], 'AS', func='median', steps_in_period=365, threshold=0.1)
#Add decimal date
y_dat['decimal_date']=y_dat['decimal_date']=y_dat.index.year+ (y_dat.index.dayofyear -1)/365

#y_dat.loc['1978':'1981', 'Precipitation']=pd.np.nan #see what happens w/o 1980 PDO bump

#Simple  OLS plot
sns.set(rc={'figure.figsize':(6, 6)}, style='whitegrid') #set seaborn figure size to A4 paper
fig=plot_funcs.OLS_plot('decimal_date', var, dat=y_dat.dropna(subset=[var]), title='Precip Adjustment Ratio', aspect=2.5)
# if save_plots:
#     plt.savefig('./figs/trends/' + Glacier + Station+ '_trendInThePrecipAdjustmentFactor'+ var +'.jpg')

In [ ]:
y_dat.tail()

In [ ]:
sns.set(rc={'figure.figsize':(6, 6)}, style='whitegrid') #set seaborn figure size to A4 paper
plt.figure()
dat_adj.ratio.plot(kind='hist', bins=50)
plt.title("Histogram of Wind Adjustment Factors")

In [ ]:
y_dat.dropna()

In [ ]:
#Aggregate to yearly data
var='Precipitation_undercatch_adj'
y_dat=pd.DataFrame()
y_dat[var]=wx.aggregate_time_with_threshold(dat_adj[var], 'AS', func='sum', steps_in_period=365)
#Add decimal date
y_dat['decimal_date']=y_dat['decimal_date']=y_dat.index.year+ (y_dat.index.dayofyear -1)/365

#y_dat.loc['1978':'1981', 'Precipitation']=pd.np.nan #see what happens w/o 1980 PDO bump

#Simple  OLS plot
sns.set(rc={'figure.figsize':(12, 6)}, style='whitegrid') #set seaborn figure size to A4 paper
fig=plot_funcs.OLS_plot('decimal_date', var, dat=y_dat.dropna(subset=[var]), title='Total Annual Precip', aspect=2.5)
if save_plots:
    plt.savefig('./figs/trends/' + Glacier + Station+ '_OLS_WindAdjustedYearly'+ var +'.jpg')

In [ ]:
save_plots

In [ ]:
dat.head()

In [ ]:
#Something is messed up in this block; must be mulitplying by scale factor a few times or something? Confusing.

# dat['Stage_Adj']=dat.Stage_Precip*scale_factor #digital data, 2003 -> present
# GUI_dat['precip_original']=GUI_dat.precip_original*scale_factor #pre-digital era

# dat[precip_cols+ ['Stage_Adj', 'precip_original']][start_good_overlap_precip_date:].cumsum().plot()

# ### Combine the 2 gages into a continous record of digital time (2003 -> present)
# dat['Precipitation']=dat['TPG_Precip']
# dat.loc[dat.index<start_good_overlap_precip_date, 'Precipitation']=pd.np.nan #set TPG records before final wind screen to NAN

# dat.loc[dat.Precipitation.isnull(), 'Precipitation']=dat.Stage_Adj #this is digital-era scaled precip

# #Plot to ensure that full digital precip record is captured as we want
# plt.figure()
# raw_data=(raw_dat.StageCumulative*1000*5*scale_factor)
# ax=raw_data.plot(color='grey')
# dat[['Precipitation', 'TPG_Precip', 'Stage_Precip', 'Stage_Adj']].cumsum().plot(ax=ax)

# plt.ylim(-10, 25000)

# Merge older data with new 15-min derived daily data
# dat=GUI_dat[['temp_original', 'precip_original']].merge(dat[[temp_col]+ precip_cols+['Precipitation']+ [wind_col]], left_index=True, right_index=True, how='outer')

# #Create single continuous precipitation record
# dat['Precip_All']=dat['Precipitation'] #this will hold precip from 2003-09-14 (first day of digital precip) _> present, and prior records before

# use_Mayo_data=dat.Precipitation.first_valid_index()-datetime.timedelta(days=1)
# dat.loc[:use_Mayo_data, "Precip_All"]=dat.loc[:use_Mayo_data, 'precip_original']